In [7]:
import numpy as np
import pandas as pd
from constants.layoutType import LayoutType
from constants.behaviourType import BehaviourType

behaviourNames = [
    behaviour.name for behaviour in BehaviourType.getExperimentBehaviourType()
]
coverageColumns = ["total50", "total75", "total90", "total100"]
methodsLabels = [
    "Wandering",
    "Greedy",
    "Distributed Frontier-based",
    "Distributed Frontier-based greedy",
    "Distributed Frontier-based assistance",
    "Role system: Scout",
    "Centralised Frontier-based",
    "Zone-grid split",
    "Zone-Voronoi",
]


In [8]:
import plotly.express as px


# Overall Sucess Rate
def getSuccessRate(df: pd.DataFrame, columnName):
    # Unsuccess to reach target corverage value = -1
    return len(df[df[columnName] >= 0]) / len(df) * 100


# Concat all agents size, layout
frames = []
for num in [15, 10, 5]:
    for layout in LayoutType.getExperimentLayout():
        df = pd.read_csv(
            f"analysis/experimentResults/experiment_{layout.name}_{num}.csv"
        )
        frames.append(df)

df_concat = pd.concat(frames)

with open("analysis/success_rate.csv", "w") as f:
    print(f"Method,50%,75%,90%,100%")
    f.write("Method,50%,75%,90%,100%")
    f.write("\n")
    for behaviourName in behaviourNames:
        df_behav = df_concat[df_concat["Behaviour"] == behaviourName]
        successRate50 = getSuccessRate(df_behav, "total50")
        successRate75 = getSuccessRate(df_behav, "total75")
        successRate90 = getSuccessRate(df_behav, "total90")
        successRate100 = getSuccessRate(df_behav, "total100")
        print(
            f"{behaviourName},{successRate50:.2f}%,{successRate75:.2f}%,{successRate90:.2f}%,{successRate100:.2f}%"
        )
        f.write(
            f"{behaviourName},{successRate50:.2f},{successRate75:.2f},{successRate90:.2f},{successRate100:.2f}"
        )
        f.write("\n")

df = pd.read_csv(f"analysis/success_rate.csv")
df_melted = df.melt(id_vars=["Method"], var_name="Coverage", value_name="Rating(%)")
fig = px.bar(df_melted, x="Method", y="Rating(%)", color="Coverage", barmode="group")
fig.update_layout(
        title=f"Coverage Success Rates",
        xaxis_title="Methods",
        xaxis=dict(
            tickmode="array",
            tickvals=[*range(len(behaviourNames))],
            ticktext=methodsLabels,
        ),
    )

fig.show()

Method,50%,75%,90%,100%
WANDERING,100.00%,97.78%,88.70%,51.85%
GREEDY,100.00%,99.44%,98.15%,88.52%
FRONTIER,100.00%,100.00%,100.00%,98.70%
GREEDY_FRONTIER,100.00%,100.00%,100.00%,97.04%
FRONTIER_ASSIST,100.00%,100.00%,100.00%,98.89%
SCOUT,100.00%,99.81%,99.81%,99.07%
FRONTIER_CENTRAL_GREEDY,100.00%,100.00%,100.00%,91.85%
ZONE_SPLIT,100.00%,100.00%,99.26%,92.22%
ZONE_VORONOI,100.00%,100.00%,100.00%,100.00%


In [9]:
import plotly.express as px


# Sucess Rate
def getSuccessRate(df: pd.DataFrame, columnName):
    # Unsuccess to reach target corverage value = -1
    return len(df[df[columnName] >= 0]) / len(df) * 100


# Concat all agents size, layout
for num in [15, 10, 5]:
    frames = []
    df_concat = []
    for layout in LayoutType.getExperimentLayout():
        df = pd.read_csv(
            f"analysis/experimentResults/experiment_{layout.name}_{num}.csv"
        )
        frames.append(df)

    df_concat = pd.concat(frames)

    with open(f"analysis/success_rate_{num}.csv", "w") as f:
        print(f"Method,50%,75%,90%,100%")
        f.write("Method,50%,75%,90%,100%")
        f.write("\n")
        for behaviourName in behaviourNames:
            df_behav = df_concat[df_concat["Behaviour"] == behaviourName]
            successRate50 = getSuccessRate(df_behav, "total50")
            successRate75 = getSuccessRate(df_behav, "total75")
            successRate90 = getSuccessRate(df_behav, "total90")
            successRate100 = getSuccessRate(df_behav, "total100")
            print(
                f"{behaviourName},{successRate50:.2f}%,{successRate75:.2f}%,{successRate90:.2f}%,{successRate100:.2f}%"
            )
            f.write(
                f"{behaviourName},{successRate50:.2f},{successRate75:.2f},{successRate90:.2f},{successRate100:.2f}"
            )
            f.write("\n")

    df = pd.read_csv(f"analysis/success_rate_{num}.csv")
    df_melted = df.melt(id_vars=["Method"], var_name="Coverage", value_name="Rating(%)")
    fig = px.bar(
        df_melted, x="Method", y="Rating(%)", color="Coverage", barmode="group"
    )
    fig.update_layout(
        title=f"Coverage Success Rates ({num} agents)",
        xaxis_title="Methods",
        xaxis=dict(
            tickmode="array",
            tickvals=[*range(len(behaviourNames))],
            ticktext=methodsLabels,
        ),
    )

    fig.show()

Method,50%,75%,90%,100%
WANDERING,100.00%,100.00%,99.44%,78.89%
GREEDY,100.00%,100.00%,100.00%,97.22%
FRONTIER,100.00%,100.00%,100.00%,97.78%
GREEDY_FRONTIER,100.00%,100.00%,100.00%,97.22%
FRONTIER_ASSIST,100.00%,100.00%,100.00%,98.33%
SCOUT,100.00%,100.00%,100.00%,98.33%
FRONTIER_CENTRAL_GREEDY,100.00%,100.00%,100.00%,87.78%
ZONE_SPLIT,100.00%,100.00%,99.44%,87.22%
ZONE_VORONOI,100.00%,100.00%,100.00%,100.00%


Method,50%,75%,90%,100%
WANDERING,100.00%,100.00%,98.33%,55.56%
GREEDY,100.00%,100.00%,98.89%,93.89%
FRONTIER,100.00%,100.00%,100.00%,100.00%
GREEDY_FRONTIER,100.00%,100.00%,100.00%,95.00%
FRONTIER_ASSIST,100.00%,100.00%,100.00%,100.00%
SCOUT,100.00%,99.44%,99.44%,98.89%
FRONTIER_CENTRAL_GREEDY,100.00%,100.00%,100.00%,92.22%
ZONE_SPLIT,100.00%,100.00%,98.33%,90.00%
ZONE_VORONOI,100.00%,100.00%,100.00%,100.00%


Method,50%,75%,90%,100%
WANDERING,100.00%,93.33%,68.33%,21.11%
GREEDY,100.00%,98.33%,95.56%,74.44%
FRONTIER,100.00%,100.00%,100.00%,98.33%
GREEDY_FRONTIER,100.00%,100.00%,100.00%,98.89%
FRONTIER_ASSIST,100.00%,100.00%,100.00%,98.33%
SCOUT,100.00%,100.00%,100.00%,100.00%
FRONTIER_CENTRAL_GREEDY,100.00%,100.00%,100.00%,95.56%
ZONE_SPLIT,100.00%,100.00%,100.00%,99.44%
ZONE_VORONOI,100.00%,100.00%,100.00%,100.00%


In [10]:
# Detailed Sucess Rate
def getSuccessRate(df: pd.DataFrame, columnName):
    # Unsuccess to reach target corverage value = -1
    return len(df[df[columnName] >= 0]) / len(df) * 100


with open("analysis/result_coverage.csv", "w") as f:
    # Table header
    f.write(
        ",".join(
            [
                "num",
                "layout",
                "Method",
                "Coverage 50%",
                "Coverage 75%",
                "Coverage 90%",
                "Coverage 100%",
            ]
        )
    )
    f.write("\n")

    # Concat all agents size, layout
    frames = []
    for num in [15, 10, 5]:
        for layout in LayoutType.getExperimentLayout():
            df = pd.read_csv(f"analysis/experimentResults/experiment_{layout.name}_{num}.csv")
            frames.append(df)

            df_concat = pd.concat(frames)

            print(f"num,Method,50%,75%,90%,100%")
            for behaviourName in behaviourNames:
                df_behav = df_concat[df_concat["Behaviour"] == behaviourName]
                successRate50 = getSuccessRate(df_behav, "total50")
                successRate75 = getSuccessRate(df_behav, "total75")
                successRate90 = getSuccessRate(df_behav, "total90")
                successRate100 = getSuccessRate(df_behav, "total100")
                print(
                    f"{num},{layout.name},{behaviourName},{successRate50:.2f}%,{successRate75:.2f}%,{successRate90:.2f}%,{successRate100:.2f}%"
                )
                f.write(f"{num},{layout.name},{behaviourName},{successRate50:.2f}%,{successRate75:.2f}%,{successRate90:.2f}%,{successRate100:.2f}%")
                f.write("\n")

num,Method,50%,75%,90%,100%
15,PLAIN,WANDERING,100.00%,100.00%,100.00%,100.00%
15,PLAIN,GREEDY,100.00%,100.00%,100.00%,100.00%
15,PLAIN,FRONTIER,100.00%,100.00%,100.00%,100.00%
15,PLAIN,GREEDY_FRONTIER,100.00%,100.00%,100.00%,100.00%
15,PLAIN,FRONTIER_ASSIST,100.00%,100.00%,100.00%,100.00%
15,PLAIN,SCOUT,100.00%,100.00%,100.00%,100.00%
15,PLAIN,FRONTIER_CENTRAL_GREEDY,100.00%,100.00%,100.00%,100.00%
15,PLAIN,ZONE_SPLIT,100.00%,100.00%,100.00%,100.00%
15,PLAIN,ZONE_VORONOI,100.00%,100.00%,100.00%,100.00%
num,Method,50%,75%,90%,100%
15,OBSTACLES,WANDERING,100.00%,100.00%,100.00%,97.50%
15,OBSTACLES,GREEDY,100.00%,100.00%,100.00%,100.00%
15,OBSTACLES,FRONTIER,100.00%,100.00%,100.00%,100.00%
15,OBSTACLES,GREEDY_FRONTIER,100.00%,100.00%,100.00%,100.00%
15,OBSTACLES,FRONTIER_ASSIST,100.00%,100.00%,100.00%,100.00%
15,OBSTACLES,SCOUT,100.00%,100.00%,100.00%,100.00%
15,OBSTACLES,FRONTIER_CENTRAL_GREEDY,100.00%,100.00%,100.00%,100.00%
15,OBSTACLES,ZONE_SPLIT,100.00%,100.00%,100.00%,90.00%
15,OBS

In [11]:
# Detailed Moves mean, std
means = {}
stds = {}

with open("analysis/moves_mean_std.csv", "w") as f:
    f.write(",".join(["num", "layout", "percent"] + behaviourNames))
    f.write("\n")

    for num in [15, 10, 5]:
        for layout in LayoutType.getExperimentLayout():
            df = pd.read_csv(f"analysis/experimentResults/experiment_{layout.name}_{num}.csv")
            df = df.replace(-1, np.nan)

            for coverage in coverageColumns:
                means[coverage] = []
                stds[coverage] = []
                for behaviourName in behaviourNames:
                    df_behav = df[df["Behaviour"] == behaviourName]
                    means[coverage].append(df_behav[coverage].mean())
                    stds[coverage].append(df_behav[coverage].std())

                row = ",".join(
                    [str(num), str(layout.name), coverage]
                    + [
                        f"{meanI:.2f}±{stdI:.2f}"
                        for meanI, stdI in zip(means[coverage], stds[coverage])
                    ]
                )

                f.write(row)
                f.write("\n")

In [12]:
import plotly.express as px

# Detailed Moves relative mean, std
frames = []

with open("analysis/moves_relative_mean_std.csv", "w") as f:
    f.write(",".join([f"Method,50%,75%,90%,100%"]))
    f.write("\n")

    for num in [15, 10, 5]:
        for layout in [l for l in LayoutType.getExperimentLayout()]:
            df = pd.read_csv(f"analysis/experimentResults/experiment_{layout.name}_{num}.csv")
            df = df.replace(-1, np.nan)

            df_normalized = df.copy()
            df_normalized[coverageColumns] = df[coverageColumns].div(
                max(df[coverageColumns].max())
            )
            frames.append(df_normalized)

    df_concat = pd.concat(frames)
    df_concat.to_csv('analysis/moves_relative.csv')

    for behaviourName in behaviourNames:
        df_behav = df_concat[df_concat["Behaviour"] == behaviourName]

        mins = []
        meds = []
        maxs = []

        for coverage in coverageColumns:
            mins.append(df_behav[coverage].min())
            meds.append(df_behav[coverage].median())
            maxs.append(df_behav[coverage].max())

        row = ",".join(
            [behaviourName]
            + [
                f"{minI:.2f}:{medI:.2f}:{maxI:.2f}"
                for minI, medI, maxI in zip(mins, meds, maxs)
            ]
        )
        print(row)

        f.write(row)
        f.write("\n")

WANDERING,0.03:0.14:1.00,0.05:0.29:1.00,0.13:0.85:1.00,0.13:0.53:1.00
GREEDY,0.01:0.03:1.00,0.01:0.07:1.00,0.02:0.19:1.00,0.02:0.16:0.99
FRONTIER,0.01:0.02:0.08,0.01:0.03:0.14,0.01:0.04:1.00,0.01:0.04:0.16
GREEDY_FRONTIER,0.01:0.02:0.08,0.01:0.03:0.13,0.01:0.04:1.00,0.01:0.04:0.54
FRONTIER_ASSIST,0.01:0.02:0.09,0.01:0.03:0.17,0.01:0.04:1.00,0.01:0.04:0.19
SCOUT,0.01:0.02:1.00,0.01:0.03:0.17,0.01:0.05:1.00,0.01:0.05:0.21
FRONTIER_CENTRAL_GREEDY,0.01:0.02:0.07,0.01:0.03:0.10,0.01:0.05:1.00,0.01:0.04:0.23
ZONE_SPLIT,0.01:0.03:0.07,0.01:0.05:1.00,0.03:0.08:1.00,0.03:0.07:0.66
ZONE_VORONOI,0.01:0.02:0.07,0.01:0.03:0.11,0.01:0.04:0.14,0.02:0.05:0.15


In [13]:
import plotly.express as px

# Plot moves relative boxplot



def plot(df):
    fig = px.box(
        df,
        x="Behaviour",
        y="Moves (Normalised)",
        color="Coverage",
        category_orders={
            "Behaviour": behaviourNames,
            "Coverage": ["50%", "75%", "90%", "100%"],
        },
    )

    fig.update_layout(
        title=f"Relative Moves",
        xaxis_title="Methods",
        xaxis=dict(
            tickmode="array",
            tickvals=[*range(len(behaviourNames))],
            ticktext=methodsLabels,
        ),
    )

    fig.show()


df = pd.read_csv(f"analysis/moves_relative.csv")

df_melted = df.melt(
    id_vars=["Behaviour", "Round"], var_name="Coverage", value_name="Moves (Normalised)"
)
df_melted["Coverage"] = df_melted["Coverage"].map(
    {
        "total50": "50%",
        "total75": "75%",
        "total90": "90%",
        "total100": "100%",
    }
)

plot(df_melted)

In [14]:
import plotly.express as px
import numpy as np

behaviourIndependentNames = [
    behaviour.name
    for behaviour in BehaviourType.getExperimentBehaviourType()
    if behaviour.value < 2
]
behaviourCoordinationNames = [
    behaviour.name
    for behaviour in BehaviourType.getExperimentBehaviourType()
    if behaviour.value >= 2
]

behaviourIndependentLabels = ["Wandering", "Greedy"]
behaviourCoordinationLabels = [
    "Distributed Frontier-based",
    "Distributed Frontier-based greedy",
    "Distributed Frontier-based assistance",
    "Role system: Scout",
    "Centralised Frontier-based",
    "Zone-grid split",
    "Zone-Voronoi",
]

labels = [behaviourIndependentLabels, behaviourCoordinationLabels]
count = 1


def plot(count, df, strategy, behaviourNames, behaviourLabels):
    fig = px.box(
        df,
        x="Behaviour",
        y="Moves",
        color="Coverage",
        category_orders={
            "Behaviour": behaviourNames,
            "Coverage": ["50%", "75%", "90%", "100%"],
        },
    )

    fig.update_layout(
        title=f"{layout.name} - {num} agents - {strategy}",
        xaxis_title="Methods",
        xaxis=dict(
            tickmode="array",
            tickvals=[*range(len(behaviourNames))],
            ticktext=behaviourLabels,
        ),
    )

    # fig.show()
    fig.write_image(f"analysis/plots/{count}_{num}_{layout.name}_{strategy}.jpeg")


for num in [15, 10, 5]:
    for layout in [
        l for l in LayoutType.getExperimentLayout() if l.value  # in [1, 2, 9]
    ]:
        df = pd.read_csv(
            f"analysis/experimentResults/experiment_{layout.name}_{num}.csv"
        )
        df = df.replace(-1, np.nan)

        for i, methods in enumerate(
            [behaviourIndependentNames, behaviourCoordinationNames]
        ):
            df_filtered = df[df["Behaviour"].isin(methods)]
            df_filtered_clean = df_filtered.copy()

            for method in methods:
                df_filtered_method = df_filtered[df_filtered["Behaviour"] == method]
                for column in coverageColumns:
                    q1 = df_filtered_method[column].quantile(0.25)
                    q3 = df_filtered_method[column].quantile(0.75)

                    df_filtered_clean[df_filtered["Behaviour"] == method] = (
                        df_filtered_method[
                            (df_filtered[column] > q1) & (df_filtered[column] < q3)
                        ]
                    )

            df_melted = df_filtered_clean.melt(
                id_vars=["Behaviour", "Round"], var_name="Coverage", value_name="Moves"
            )
            df_melted["Coverage"] = df_melted["Coverage"].map(
                {
                    "total50": "50%",
                    "total75": "75%",
                    "total90": "90%",
                    "total100": "100%",
                }
            )

            plot(
                count, df_melted, ["Independent", "Coordination"][i], methods, labels[i]
            )
            count += 1

/var/folders/j7/7rsvn_zn315974xz5qcwhjrw0000gn/T/ipykernel_31517/871256084.py:78: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/j7/7rsvn_zn315974xz5qcwhjrw0000gn/T/ipykernel_31517/871256084.py:78: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/j7/7rsvn_zn315974xz5qcwhjrw0000gn/T/ipykernel_31517/871256084.py:78: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/j7/7rsvn_zn315974xz5qcwhjrw0000gn/T/ipykernel_31517/871256084.py:78: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/j7/7rsvn_zn315974xz5qcwhjrw0000gn/T/ipykernel_31517/871256084.py:78: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/j7/7rsvn_zn315974xz5qcwhjrw0000gn/T/ipykernel_31517/871256084.py:78: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/j7/7rsvn_zn315974xz5qcwhjrw